# Topic Modeling

#### Evan Yathon

This notebook is intended to be run with papermill from the project root.

The purpose of this notebook is to process the review title and content to extract key phrases in each review.  Then key phrases will be used in a regression analysis to find out what is most important for a reviewer in recommending or not recommending the airline.

Topic modeling using LDA will be the tool of choice.  Probability of belonging to a certain topic will be new features used in the regression analysis.

Usage:

`papermill src/ipynbs/topic_modeling.ipynb src/topic_modeling_ran.ipynb -p load_path data/cleaned_gw_reviews.csv -p save_path data/topic_modeling_gw_reviews.csv`

In [1]:
#parameters section for Papermill

load_path = "../../data/cleaned_gw_reviews.csv"
save_path = "../../data/topic_modeling_gw_reviews.csv"

In [2]:
# Parameters
load_path = "data/cleaned_gw_reviews.csv"
save_path = "data/topic_modelling_gw_reviews.csv"


In [3]:
# loading packages

# utils
import pandas as pd
import re
import numpy as np

# NLP
import spacy
import gensim 
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim import models
import pyLDAvis.gensim

# display preference for ipynbs
%matplotlib inline

In [4]:
# ensure the dates are parsed correctly with parse_dates argument
reviews = pd.read_csv(load_path, parse_dates = ["date_of_review", "date_flown"])

In [5]:
reviews.head()

,title,review_value,n_user_reviews,reviewer_name,reviewer_country,date_of_review,review_text,aircraft,traveller_type,seat_type,route,date_flown,seat_comfort_rating,cabin_staff_service_rating,food_and_beverages_rating,inflight_entertainment_rating,ground_service_rating,value_for_money_rating,recommendation
0,"""Seat was fine with enough legroom""",7.0,9 reviews,Sander van Kan,Netherlands,2019-07-01,✅ Trip Verified | Dusseldorf to Berlin. Eurowi...,A319,Couple Leisure,Economy Class,Dusseldorf to Berlin,2019-06-01,4.0,3.0,NaN,1.0,3.0,5.0,yes
1,"""crew were smiling and good""",6.0,9 reviews,Sander van Kan,Netherlands,2019-07-01,✅ Trip Verified | Berlin to Dusseldorf. Eurowi...,A319,Couple Leisure,Economy Class,Berlin to Dusseldorf,2019-06-01,3.0,3.0,NaN,1.0,3.0,5.0,yes
2,"""only two agents available""",1.0,6 reviews,Andrew Maynard,United Kingdom,2017-01-04,Check in process at Cologne very poor. Flight ...,NaN,Couple Leisure,Economy Class,CGN to MAN,2017-01-01,2.0,2.0,NaN,NaN,1.0,2.0,no
3,"""good flight and friendly staff""",7.0,1 reviews,T Steen,Netherlands,2016-09-13,✅ Verified Review | Amsterdam to Stuttgart. G...,NaN,Business,Economy Class,AMS to STR,2016-09-01,5.0,5.0,1.0,NaN,5.0,5.0,yes
4,"""never been treated as badly""",1.0,NaN,Karen Kirner,Austria,2016-08-16,✅ Verified Review | I have been a frequent tr...,NaN,Business,Economy Class,DUS to VIE,2016-08-01,1.0,1.0,NaN,NaN,3.0,1.0,no


### Topic Modeling - Preprocessing

Prior to LDA, preprocessing of the text is necessary.  We need to:
- Tokenize each document (review)
- Minimum token length
- Remove stopwords
- Lemmatize
- Consider only specific parts of speech (nouns, verbs etc.)

Use [`spaCy`](https://spacy.io/) for this.

In [6]:
# load the english spacy model
eng_nlp = spacy.load('en')

In [7]:
# create a function to preprocess the reviews and review titles

def preprocess_text(text, min_token_len = 2, relevant_pos = ["NOUN", "VERB", "ADJ"]):
    """
    Given text, min_token_len, and relevant_pos carry out preprocessing of the text 
    and return a preprocessed string. 
    
    Keyword arguments:
    text (str): the text to be preprocessed
    min_token_len (int): min_token_length required
    relevant_pos (list): a list of relevant pos tags
    
    Returns: (str) the preprocessed text
    """
    # use several regex expressions to preprocess strange characters
    # or unwanted text
    
    # remove verified emoji
    text = re.sub(r"✅", "", text)
    
    # remove verified review and trip verified
    text = re.sub(r"Verified Review|Trip Verified", "", text)
    
    # remove anything that is not a word
    text = re.sub(r"[^\w]", " ", text)
    
    # replace multiple spaces with a single space
    text = re.sub(r"\s+", " ", text)
    
    # remove numbers
    text = re.sub(r"[0-9]+", "", text)
    
    # change all text to lowercase
    text = text.lower()
    
    # remove common terms found in topic modeling output
    # eg/ germanwings, flight
    # these words are not useful for this application as they
    # are common terms found in most reviews, and won't be indicative of 
    # things that influenced their recommendation
    terms = ["flight", "germanwing", "fly", "review"]
    text = re.sub("|".join(terms)," ", text)
    
    # utilize spaCy to tokenize and break text into lemmas/POS:
    # - remove stopwords
    # - reduce words to their lemmas
    # - only keep relevant parts of speech
    # - remove short words less than min_token_len
    
    # spacy will utilize
    doc = eng_nlp(text)
    
    processed_text = ""
    
    for token in doc:
        if not token.is_stop and len(token.text) >= min_token_len and token.pos_ in relevant_pos:
            processed_text += " " + token.lemma_
    
    return processed_text
    

Use the preprocessing function to process all review texts and review titles.

A good check will be to see if the review topics somewhat make sense with the title topics

In [8]:
reviews["clean_review_text"] = reviews["review_text"].apply(preprocess_text, min_token_len = 3)
reviews["clean_title"] = reviews["title"].apply(preprocess_text, min_token_len = 3)

In [9]:
reviews[["clean_review_text", "review_text", "clean_title", "title"]].head(10)

,clean_review_text,review_text,clean_title,title
0,dusseldorf berlin eurowing operate slight del...,✅ Trip Verified | Dusseldorf to Berlin. Eurowi...,seat fine legroom,"""Seat was fine with enough legroom"""
1,berlin dusseldorf eurowing operate different ...,✅ Trip Verified | Berlin to Dusseldorf. Eurowi...,crew smile good,"""crew were smiling and good"""
2,check process cologne poor display delay minu...,Check in process at Cologne very poor. Flight ...,agent available,"""only two agents available"""
3,amsterdam stuttgart good friendly staff airpl...,✅ Verified Review | Amsterdam to Stuttgart. G...,good friendly staff,"""good flight and friendly staff"""
4,frequent traveler year fly distance cabin cla...,✅ Verified Review | I have been a frequent tr...,treat,"""never been treated as badly"""
5,fly london stanste cologne bonn delay hour lo...,✅ Verified Review | Flew Germanwings from Lon...,cramp,"""very cramped"""
6,boyfriend book basic fare hamburg rome fiumic...,My boyfriend and I booked the basic fare from ...,staff friendly,"""staff were friendly"""
7,operation transfer take eurowing read eurowin...,Germanwings operations have now been transferr...,refer eurowing,PLEASE REFER TO EUROWINGS
8,operate barcelona berlin tegel fight time qui...,Lufthansa operated by Germanwings from Barcelo...,great value money,"""great value for money"""
9,bangkok london cologne imagine ryanair hour f...,Germanwings from Bangkok to London via Cologne...,pay little money,"""pay very little money"""


It looks mostly good, but I noticed that in the first review it is removing 'enough'.  This could be valuable in context, but I'll keep digging for now.  It's an issue since 'enough' is a stopword and is being removed.

### Topic Modeling: Dictionary and Document-Term Co-occurrence Matrix

In [10]:
# change the cleaned reviews and cleaned titles to 
# an array, required by corpora.Dictionary
review_corpus = [doc.split() for doc in reviews["clean_review_text"].tolist()]
title_corpus = [doc.split() for doc in reviews["clean_title"].tolist()]

In [11]:
# use gensim dictionary to map between words and integer ids
dct_review = corpora.Dictionary(review_corpus)
dct_title = corpora.Dictionary(title_corpus)

In [12]:
# create the document-term co-occurrence matrices
doc_term_mat_review = [dct_review.doc2bow(doc) for doc in review_corpus]
doc_term_mat_title = [dct_title.doc2bow(doc) for doc in title_corpus]

### Topic Modeling:  LDA Topic Model

Number of topics was tuned using the visualizations below by examining the Intertopic distance map.  When there was no overlap and the topics were separated by a fair amount, the number of topics was chosen.  Number of topics was also kept perhaps artifically low, as I wanted to have fewer features in regression due to the low sample size.

In addition, trial and error to select hyperparameters was used until topics were generated that made sense.

In [13]:
# create the LDA models
# use a lower alpha, meaning that each document is representative of only a few topics
# do this so that each review doesn't contain similar topics and will have variation
# that we can use to figure out why the recommendation was a yes or no

n_review_topics = 3
n_title_topics = 2

lda_review = models.LdaModel(corpus = doc_term_mat_review, id2word = dct_review, num_topics = n_review_topics, 
                             alpha = 0.5, eta = 0.01, passes = 5, random_state = 40)
lda_title = models.LdaModel(corpus = doc_term_mat_title, id2word = dct_title, num_topics = n_title_topics, 
                            alpha = 0.9, eta = 0.1, passes = 5, random_state = 40)

In [14]:
# view the topics and the words that describe them
lda_review.print_topics()

[(0,
  '0.023*"check" + 0.021*"seat" + 0.017*"time" + 0.014*"good" + 0.012*"service" + 0.011*"return" + 0.010*"staff" + 0.010*"bag" + 0.009*"luggage" + 0.009*"delay"'),
 (1,
  '0.017*"passenger" + 0.017*"time" + 0.016*"delay" + 0.014*"hour" + 0.013*"minute" + 0.012*"return" + 0.012*"staff" + 0.011*"airline" + 0.009*"tell" + 0.009*"pay"'),
 (2,
  '0.021*"seat" + 0.021*"good" + 0.017*"crew" + 0.017*"drink" + 0.015*"cologne" + 0.012*"price" + 0.012*"snack" + 0.012*"service" + 0.012*"board" + 0.012*"time"')]

In [15]:
# view the topics and the words that describe them
lda_title.print_topics()

[(0,
  '0.798*"customer" + 0.013*"money" + 0.011*"great" + 0.011*"refer" + 0.011*"don" + 0.011*"late" + 0.011*"value" + 0.011*"eurowing" + 0.011*"refuse" + 0.010*"water"'),
 (1,
  '0.422*"customer" + 0.082*"friendly" + 0.062*"staff" + 0.041*"crew" + 0.031*"cabin" + 0.022*"delay" + 0.021*"little" + 0.021*"professional" + 0.020*"good" + 0.012*"seat"')]

From the review topics, here are my interpretations of what they are about:

| Review Text Topic Number | Interpretation          |
|--------------------------|-------------------------|
| 0                        | luggage and seats       |
| 1                        | delays and time         |
| 2                        | food beverages and crew |

And for the title topics:

| Title Text Topic Number 	| Interpretation   	|
|-------------------------	|------------------	|
| 0                       	| money value      	|
| 1                       	| staff and delays 	|


Note that for the title topics, there aren't as many words.  Maybe they should have been rolled into the reviews as well, but I wanted to give them their own separate meaning, as the title often summarizes and emphasizes the review feeling.

In [16]:
# assign the topics to a list for later use

review_topic_interp = ["review_luggage_seats", "review_time_delays", "review_food_bev_crew"]
title_topic_interp = ["title_money_value" , "title_staff_delays"]

### Topic Modeling: Visualization

In [17]:
# create the visualizations and save html to a folder for viewing without jupyter notebook
vis_reviews = pyLDAvis.gensim.prepare(lda_review, doc_term_mat_review, dct_review, sort_topics = False)
pyLDAvis.save_html(vis_reviews, "img/vis_reviews.html")

vis_titles = pyLDAvis.gensim.prepare(lda_title, doc_term_mat_title, dct_title, sort_topics = False)
pyLDAvis.save_html(vis_titles, "img/vis_titles.html")

C:\Users\evany\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


C:\Users\evany\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


Links to the visualizations are below.  Note that they will need to be viewed locally, as GitHub won't render `html` files properly at this time.

[Link to saved `html` of the reviews visualization](https://github.com/EvanYathon/airline_reviews/blob/master/img/vis_reviews.html)

[Link to saved `html` of the titles visualization](https://github.com/EvanYathon/airline_reviews/blob/master/img/vis_titles.html)

In [18]:
# view the reviews if running Jupyter Notebook
pyLDAvis.enable_notebook()
vis_reviews

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.001790 -0.118210       1        1  33.008957
1     -0.122036  0.060396       2        1  31.810266
2      0.123826  0.057814       3        1  35.180779, topic_info=    Category       Freq         Term      Total  loglift  logprob
4    Default  44.000000        drink  44.000000  30.0000  30.0000
29   Default  69.000000        check  69.000000  29.0000  29.0000
71   Default  34.000000        snack  34.000000  28.0000  28.0000
38   Default  48.000000         hour  48.000000  27.0000  27.0000
10   Default  27.000000      legroom  27.000000  26.0000  26.0000
9    Default  77.000000         good  77.000000  25.0000  25.0000
311  Default  20.000000  information  20.000000  24.0000  24.0000
110  Default  12.000000         long  12.000000  23.0000  23.0000
42   Default  55.000000    passenger  55.000000  22.0000  22.0000
14   Default  96.000000         seat  96.000000  21.0000  21.0000
192  Default  13.000000      include  13.000000  20.0000  20.0000
266  Default  23.000000    efficient  23.000000  19.0000  19.0000
159  Default  23.000000  comfortable  23.000000  18.0000  18.0000
176  Default  32.000000         tell  32.000000  17.0000  17.0000
2    Default  56.000000         crew  56.000000  16.0000  16.0000
104  Default   8.000000         high   8.000000  15.0000  15.0000
127  Default  22.000000         wait  22.000000  14.0000  14.0000
399  Default   7.000000    technical   7.000000  13.0000  13.0000
67   Default  24.000000     sandwich  24.000000  12.0000  12.0000
70   Default  22.000000        smart  22.000000  11.0000  11.0000
128  Default  31.000000          bag  31.000000  10.0000  10.0000
64   Default  37.000000     friendly  37.000000   9.0000   9.0000
156  Default  22.000000        clean  22.000000   8.0000   8.0000
401  Default   6.000000        train   6.000000   7.0000   7.0000
374  Default   9.000000     possible   9.000000   6.0000   6.0000
97   Default   9.000000         bonn   9.000000   5.0000   5.0000
91   Default  13.000000         year  13.000000   4.0000   4.0000
27   Default  15.000000       cancel  15.000000   3.0000   3.0000
798  Default   6.000000    complaint   6.000000   2.0000   2.0000
284  Default   9.000000        pitch   9.000000   1.0000   1.0000
..       ...        ...          ...        ...      ...      ...
67    Topic3  18.053053     sandwich  24.256157   0.7493  -4.7530
156   Topic3  16.669481        clean  22.210907   0.7577  -4.8327
212   Topic3   6.012702       choice   7.107038   0.8775  -5.8524
69    Topic3  14.525622        small  20.208330   0.7145  -4.9704
2     Topic3  34.715153         crew  56.303490   0.5611  -4.0991
266   Topic3  15.747833    efficient  23.079716   0.6624  -4.8896
579   Topic3   3.597667       toilet   4.093418   0.9156  -6.3660
9     Topic3  43.050819         good  77.863884   0.4521  -3.8839
64    Topic3  22.722931     friendly  37.047501   0.5558  -4.5229
118   Topic3  25.785198        price  45.013664   0.4875  -4.3965
30    Topic3  30.978992      cologne  60.092125   0.3821  -4.2130
14    Topic3  43.903969         seat  96.353256   0.2587  -3.8643
8     Topic3  16.486767         food  28.149889   0.5097  -4.8438
58    Topic3  25.271021        board  49.041710   0.3817  -4.4166
79    Topic3  20.478647        cabin  39.046051   0.3993  -4.6269
200   Topic3  14.676568        short  25.085690   0.5086  -4.9601
70    Topic3  13.343217        smart  22.958988   0.5020  -5.0553
123   Topic3  25.689610      service  58.610126   0.2198  -4.4002
139   Topic3  13.419001         free  24.057285   0.4609  -5.0496
84    Topic3  14.252327          fly  27.046642   0.4040  -4.9894
112   Topic3  15.797100          low  31.846790   0.3436  -4.8865
31    Topic3  17.555563         cost  37.882500   0.2756  -4.7809
66    Topic3  16.213470        offer  35.926338   0.2490  -4.8605
51    Topic3  25.070827         time  91.132782  -0.2459 

In [19]:
# view the reviews if running Jupyter Notebook
vis_titles

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0      0.080257  0.0       1        1  49.458412
1     -0.080257  0.0       2        1  50.541584, topic_info=   Category        Freq          Term       Total  loglift  logprob
43  Default  111.000000      customer  111.000000  30.0000  30.0000
8   Default    8.000000      friendly    8.000000  29.0000  29.0000
9   Default    6.000000         staff    6.000000  28.0000  28.0000
3   Default    4.000000          crew    4.000000  27.0000  27.0000
24  Default    3.000000         cabin    3.000000  26.0000  26.0000
30  Default    2.000000         delay    2.000000  25.0000  25.0000
17  Default    2.000000        little    2.000000  24.0000  24.0000
19  Default    2.000000  professional    2.000000  23.0000  23.0000
4   Default    2.000000          good    2.000000  22.0000  22.0000
2   Default    1.000000          seat    1.000000  21.0000  21.0000
40  Default    1.000000         offer    1.000000  20.0000  20.0000
37  Default    1.000000      terrible    1.000000  19.0000  19.0000
36  Default    1.000000          food    1.000000  18.0000  18.0000
1   Default    1.000000       legroom    1.000000  17.0000  17.0000
41  Default    1.000000      heathrow    1.000000  16.0000  16.0000
31  Default    1.000000       problem    1.000000  15.0000  15.0000
39  Default    1.000000          help    1.000000  14.0000  14.0000
38  Default    1.000000   explanation    1.000000  13.0000  13.0000
42  Default    1.000000        minute    1.000000  12.0000  12.0000
10  Default    1.000000         treat    1.000000  11.0000  11.0000
18  Default    1.000000           pay    1.000000  10.0000  10.0000
14  Default    1.000000         great    1.000000   9.0000   9.0000
0   Default    1.000000          fine    1.000000   8.0000   8.0000
13  Default    1.000000         refer    1.000000   7.0000   7.0000
32  Default    1.000000           don    1.000000   6.0000   6.0000
29  Default    1.000000       service    1.000000   5.0000   5.0000
23  Default    1.000000          info    1.000000   4.0000   4.0000
33  Default    1.000000          late    1.000000   3.0000   3.0000
16  Default    1.000000         value    1.000000   2.0000   2.0000
12  Default    1.000000      eurowing    1.000000   1.0000   1.0000
..      ...         ...           ...         ...      ...      ...
9    Topic2    5.733244         staff    6.196259   0.6047  -2.7863
30   Topic2    2.030692         delay    2.198285   0.6031  -3.8242
3    Topic2    3.812208          crew    4.190169   0.5878  -3.1944
24   Topic2    2.902899         cabin    3.192338   0.5873  -3.4669
2    Topic2    1.082441          seat    1.196486   0.5822  -4.4533
40   Topic2    1.078565         offer    1.196091   0.5789  -4.4569
37   Topic2    1.072339      terrible    1.195457   0.5737  -4.4627
36   Topic2    1.072074          food    1.195430   0.5735  -4.4630
1    Topic2    1.070119       legroom    1.195230   0.5718  -4.4648
41   Topic2    1.069619      heathrow    1.195179   0.5714  -4.4653
31   Topic2    1.067292       problem    1.194942   0.5694  -4.4674
39   Topic2    1.066045          help    1.194814   0.5683  -4.4686
38   Topic2    1.065256   explanation    1.194734   0.5677  -4.4693
42   Topic2    1.063252        minute    1.194529   0.5660  -4.4712
10   Topic2    1.063064         treat    1.194505   0.5658  -4.4714
18   Topic2    1.059732           pay    1.194169   0.5629  -4.4745
0    Topic2    1.058224          fine    1.194017   0.5616  -4.4760
17   Topic2    1.927117        little    2.187736   0.5555  -3.8765
19   Topic2    1.926006  professional    2.187623   0.5550  -3.8771
29   Topic2    1.049504       service    1.193128   0.5541  -4.4842
23   Topic2    1.046673          info    1.192838   0.5517  -4.4869
20   Topic2    1.038873       airline    1.192044   0.5448  -4.4944
4    Topic2    1.900111          good    2.184986   0.5427  -3.8906
35   Topic2    1.0283

### Topic Modeling: Assigning each Review a Topic

Create a handy function to help assign topics to the reviews dataframe

In [20]:

def get_correct_topics(lda_doc_topics, num_topics):
    """
    get_correct_topics uses an output LDA object to correctly place
    LDA outputs to their correct topic column number for use in a
    pandas dataframe
    
    Args:
        lda_doc_topics (TransformedCorpus): Document topic probabilities obtained through
                                            the .get_document_topics method of an LDA obj
        num_topics (int) : Number of topics in lda_doc_topics
    Returns:
        pandas dataframe of correctly assigned LDA topics
    """
    # placeholder array with same number of rows as the input 
    # corpus and number of topics for columns
    # initialize all zeros so that topics with missing probabilities
    # will be zero
    
    topic_prob_array = np.zeros((pd.DataFrame(lda_doc_topics).shape[0], num_topics))

    for row_num, row in enumerate(lda_doc_topics):

        # assign each topic to the correct
        # topic[0] is the topic number
        # topic[1] is the corresponding probability
        for topic in row:
            topic_prob_array[row_num, topic[0]] = topic[1]

    topic_prob_df = pd.DataFrame(topic_prob_array)
    
    return topic_prob_df

Get both title and reviews dataframes, concatenate with `reviews`

In [21]:
review_prob_df = get_correct_topics(lda_review.get_document_topics(doc_term_mat_review), n_review_topics)
title_prob_df = get_correct_topics(lda_title.get_document_topics(doc_term_mat_title), n_title_topics)

# name the columns appropriately
review_prob_df.columns = review_topic_interp
title_prob_df.columns = title_topic_interp

#concatenate with reviews
reviews_concat = pd.concat([reviews, review_prob_df, title_prob_df], axis = 1)

In [22]:
# save new dataframe with new features
reviews_concat.to_csv(save_path, index = False)